<a href="https://colab.research.google.com/github/aquibjaved/Bits_and_Pieces_DL/blob/main/positional_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz 

--2022-01-24 17:54:00--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.134.136
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.134.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  54.6MB/s    in 38s     

2022-01-24 17:54:39 (41.2 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [2]:
!gunzip /content/GoogleNews-vectors-negative300.bin.gz

In [3]:
import gensim.models.keyedvectors as word2vec
from sklearn.metrics.pairwise import cosine_similarity
import math
import numpy as np

In [4]:
path="/content/GoogleNews-vectors-negative300.bin"
model = word2vec.KeyedVectors.load_word2vec_format(path, binary=True)

In [5]:
def _get_sent_vector(sent: str):
  sent_tok = sent.split()
  vector = []
  for token in sent_tok:
    vector.append(model.get_vector(token))

  return np.mean(np.array(vector), axis=0)

In [6]:
sent_1 = _get_sent_vector("this is cat") # removed 'a' because 'a' isn't 
sent_2 = _get_sent_vector("this is dog") # removed 'a' 

In [12]:
print(f"The similarity score {cosine_similarity([sent_1], [sent_2])[0][0]}")

The similarity score 0.8870273232460022


In [13]:
def get_positional_encoding(pos):
    pos_emb = []
    d = 300
    for i in range(300):
        if i % 2 ==0:
            pos_emb.append(math.sin(pos/(10000**((2*i)/d))))
        else:
            pos_emb.append(math.cos(pos/(10000**((2*i)/d))))
    return pos_emb

In [18]:
def _get_sent_vector_with_encoding(sent: str):
  sent_tok = sent.split()
  vector = []
  for idx, token in enumerate(sent_tok):
    encoding_vector = get_positional_encoding(idx)
    word_vector = model.get_vector(token)
    vector.append(np.array(encoding_vector)+ word_vector)
  return np.mean(np.array(vector), axis=0)

In [28]:
sent_vec1 = _get_sent_vector_with_encoding(sent="this cat is black color")
sent_vec2 = _get_sent_vector_with_encoding(sent="this dog jumps over the mat")

In [29]:
cosine_similarity([sent_vec1], [sent_vec2])

array([[0.99195351]])